# The Product Cut
Thomas Laurent, James von Brecht, Xavier Bresson, Arthur Szlam

Neural Information Processing Systems (NIPS), 2016

[NIPS Official Link](https://papers.nips.cc/paper/6226-the-product-cut)

Code version: Feb 1, 2017

In [8]:
# Load libraries

# Math
import numpy as np

# Import data
import scipy.io

In [9]:
# Load 20NEWS dataset
mat = scipy.io.loadmat('datasets/20NEWS.mat')
W = mat['A']
n = W.shape[0]
Cgt = mat['C'] - 1; Cgt = Cgt.squeeze()
nc = len(np.unique(Cgt))
print(n,nc)

19934 20
